In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

24/08/07 15:48:41 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 15:48:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 15:48:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 15:48:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/07 15:48:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/07 15:48:42 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/07 15:48:42 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/08/07 15:48:42 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
data = spark.read.csv('customer_churn.csv', inferSchema= True, header = True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
data.describe().show()

24/08/07 15:49:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols = ['Age', 'Total_Purchase', 'Account_Manager',
                                         'Years', 'Num_Sites'], outputCol = 'features')

In [10]:
output = assembler.transform(data)

In [11]:
final_data = output.select ('features', 'churn')

In [12]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [13]:
lr_churn = LogisticRegression(labelCol = 'churn')

In [14]:
fitted_churn_model = lr_churn.fit(train_churn)

24/08/07 15:54:15 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [15]:
training_sum = fitted_churn_model.summary

In [16]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               638|                638|
|   mean|0.1677115987460815|0.12852664576802508|
| stddev|0.3739031240943127| 0.3349378794931142|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [18]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [19]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,0.0...|    0|[6.05373909352146...|[0.99765644295942...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.08473571196654...|[0.95625869852405...|       0.0|
|[30.0,8403.78,1.0...|    0|[5.68013301623262...|[0.99659850524186...|       0.0|
|[30.0,8874.83,0.0...|    0|[2.92689703140875...|[0.94916014934058...|       0.0|
|[30.0,10960.52,1....|    0|[2.20897921254921...|[0.90105295420813...|       0.0|
|[30.0,12788.37,0....|    0|[2.47557720287029...|[0.92241185690058...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.10008725310416...|[0.95689634402043...|       0.0|
|[31.0,7073.61,0.0...|    0|[2.80288731574184...|[0.94283164992429...|       0.0|
|[31.0,10182.6,1.0...|    0|[4.66918008440781...|[0.99070720883482...|       0.0|
|[32.0,8011.38,0

In [20]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'churn')

In [21]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [22]:
auc

0.7608049272592121

## predict on new data

In [23]:
final_lr_model = lr_churn.fit(final_data)

In [24]:
new_customers = spark.read.csv('new_customers.csv', inferSchema= True, header = True)

In [25]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [26]:
test_new_customers = assembler.transform (new_customers)

In [27]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [28]:
final_results = final_lr_model.transform(test_new_customers)

In [30]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [31]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                   6|               6|
|   mean|         NULL|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|                NULL|            NULL|
| stddev|         NULL| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|                NULL|            NULL|
|    min|Andrew Mccall|              22.0|            100.0|          